In [1]:
from rdflib import Namespace, Graph

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes

In [2]:
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://", shacl_engine="topquadrant")

# create the model w/ a namespace
BLDG = Namespace("urn:ex/")
bldg = Model.create(BLDG)
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

things = []

In [3]:
# load libraries
# NREL templates for 223P
nrel_lib = Library.load(directory="../libraries/ashrae/223p/nrel-templates")
# 223P ontology
s223 = Library.load(ontology_graph="../libraries/ashrae/223p/ontology/223p.ttl")

/Users/gabe/src/NREL/scratch/BuildingMOTIF/.venv/lib/python3.11/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [4]:
# get some templates from the library
mau_templ = nrel_lib.get_template_by_name("makeup-air-unit").inline_dependencies()
vav_templ = nrel_lib.get_template_by_name("vav-reheat").inline_dependencies()
duct_templ = nrel_lib.get_template_by_name("duct").inline_dependencies()
zone_templ = nrel_lib.get_template_by_name("hvac-zone").inline_dependencies()
space_templ = nrel_lib.get_template_by_name("hvac-space").inline_dependencies()
fcu_templ = nrel_lib.get_template_by_name("fcu").inline_dependencies()
chws_templ = nrel_lib.get_template_by_name("chilled-water-system").inline_dependencies()
hws_templ = nrel_lib.get_template_by_name("hot-water-system").inline_dependencies()

In [5]:
# create a makeup air unit
mau = mau_templ.evaluate({"name": BLDG.MAU, "air-supply": BLDG.MAU_Supply})
things.append(mau)

/Users/gabe/src/NREL/scratch/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "sa_pressure_sensor, cooling-coil-valve-in-mapsto, evaporative-cooler-evap-cool-pump-2stage-vfd-cur, supply-fan-out-mapsto, cooling-coil, evaporative-cooler, cooling-coil-air-in, evaporative-cooler-evap-cool-pump-2stage, cooling-coil-pump-onoff-cmd, oad-out, evaporative-cooler-evap-cool-pump-2stage-vfd-energy, heating-coil-water-out-mapsto, cooling-coil-pump-vfd-cur, heating-coil-water-in, heating-coil-water-out, evaporative-cooler-evap-cool-pump-2stage-onoff-cmd, evaporative-cooler-evap-cool-pump-2stage-vfd-pwr, cooling-coil-pump-onoff-sts, cooling-coil-valve-feedback, final-filter-out-mapsto, supply-fan-motor-status, evaporative-cooler-water-out, cooling-coil-pump, evaporative-cooler-evap-cool-sump-tank, evaporative-cooler-evap-cool-pump-2stage-in, pre-filter-in, cooling-coil-valve-command, pre-filter, c1, MAU-HRC-air-in-mapsto, supply-fan-in, evaporative-cooler-water-in, coo

In [6]:
# make 2 VAVs, connect them to the MAU via ducts
vav1 = vav_templ.evaluate({"name": BLDG["VAV-1"], "air-in": BLDG["VAV-1-in"]})
mau_to_vav1 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-1-in"]})
things.append(vav1)
things.append(mau_to_vav1)

vav2 = vav_templ.evaluate({"name": BLDG["VAV-2"], "air-in": BLDG["VAV-2-in"]})
things.append(vav2)
mau_to_vav2 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-2-in"]})
things.append(mau_to_vav2)

/Users/gabe/src/NREL/scratch/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "rhc-air-out, sup-air-pressure-sensor, air-out-mapsto, rhc-valve-feedback, dmp-out, rhc-valve-in, dmp-in-mapsto, rhc-supply-water-temp, sup-air-temp, rhc-air-in, sup-air-temp-sensor, rhc-valve, dmp-out-mapsto, rhc-water-in, c0, rhc, dmp-in, rhc-air-in-mapsto, sup-air-flow, rhc-valve-out, rhc-return-water-temp, dmp-air-out, rhc-valve-in-mapsto, rhc-water-out-mapsto, dmp-command, dmp-feedback, air-out, rhc-return-water-temp-sensor, rhc-air-out-mapsto, sup-air-flow-sensor, rhc-supply-water-temp-sensor, rhc-valve-out-mapsto, dmp, air-in-mapsto, rhc-valve-command, rhc-water-out, sup-air-pressure, rhc-water-in-mapsto" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/scratch/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "name" were not provided during evaluation
  warnings.warn(


In [7]:
# VAV1 goes to Zone 1
zone1 = zone_templ.evaluate({"name": BLDG["zone1"]})
things.append(zone1)
zone1space1 = space_templ.evaluate({"name": BLDG["zone1space1"], "zone": BLDG["zone1"]})
things.append(zone1space1)
vav1_to_zone1 = duct_templ.evaluate({"a": BLDG["VAV-1-out"], "b": BLDG["zone1-in"]})
things.append(vav1_to_zone1)

/Users/gabe/src/NREL/scratch/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "temp-sensor, physical-space, in2-mapsto, out2, in3, in4, in, in3-mapsto, supply-air-flow, out2-mapsto, sup-flow-sensor, temp, out-mapsto, exhaust-air-flow, in4-mapsto, exh-flow-sensor, out, in2, relative-humidity, humidity-sensor, in-mapsto" were not provided during evaluation
  warnings.warn(


In [8]:
# VAV2 goes to FCU1
fcu1 = fcu_templ.evaluate({"name": BLDG["fcu1"]})
vav2_to_fcu1 = duct_templ.evaluate({"a": BLDG["VAV-2-out"], "b": BLDG["fcu1-in"]})
things.extend([fcu1, vav2_to_fcu1])

/Users/gabe/src/NREL/scratch/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "fan-out, fan-start-cmd, cooling-coil-valve-in-mapsto, DA-temp, cooling-coil, heating-coil-water-in-mapsto, cooling-coil-pump-out-mapsto, cooling-coil-air-in, heating-coil-air-out-mapsto, in, fan-vfd-volt, heating-coil-air-out, cooling-coil-pump-onoff-cmd, zone-temp, heating-coil, heating-coil-water-out-mapsto, cooling-coil-pump-vfd-cur, heating-coil-water-in, cooling-coil-water-in-mapsto, heating-coil-water-out, fan-vfd-energy, in-mapsto, cooling-coil-pump-vfd-fb, cooling-coil-pump-onoff-sts, fan-motor-status, cooling-coil-entering-air-temp, heating-coil-return-water-temp-sensor, cooling-coil-valve-feedback, fan-out-mapsto, zone-humidity, cooling-coil-leaving-air-wetbulb-temp, heating-coil-air-in, cooling-coil-pump, cooling-coil-valve-command, cond-overflow, out-mapsto, cooling-coil-water-in, heating-coil-valve-feedback, cooling-coil-valve-in, fan-vfd-flt, cooling-coil-leaving

In [9]:
# FCU1 goes to Zone 2
zone2 = zone_templ.evaluate({"name": BLDG["zone2"]})
zone2space1 = space_templ.evaluate({"name": BLDG["zone2space1"], "zone": BLDG["zone2"]})
fcu1_to_zone2 = duct_templ.evaluate({"a": BLDG["fcu1-out"], "b": BLDG["zone2-in"]})
things.extend([zone2, zone2space1, fcu1_to_zone2])

/Users/gabe/src/NREL/scratch/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "temp-sensor, physical-space, in2-mapsto, out2, in3, in4, in, in3-mapsto, supply-air-flow, out2-mapsto, sup-flow-sensor, temp, out-mapsto, exhaust-air-flow, in4-mapsto, exh-flow-sensor, out, in2, humidity-sensor, relative-humidity, in-mapsto" were not provided during evaluation
  warnings.warn(


In [10]:
# add chws, hws
chws = chws_templ.evaluate({"name": BLDG["CHWS"]})
hws = hws_templ.evaluate({"name": BLDG["HWS"]})
things.extend([chws, hws])

/Users/gabe/src/NREL/scratch/BuildingMOTIF/buildingmotif/dataclasses/template.py:390: UserWarning: Parameters "lead-chw-pump-vfd-fb, chw-hx-chw-supply-temperature, bypass-valve-out, chw-hx-A-in, lead-chw-booster-pump-onoff-cmd, lead-chw-booster-pump-vfd-frq, chw-hx-B-out-mapsto, standby-chw-booster-pump-vfd-spd, bypass-valve-in-mapsto, standby-chw-pump-vfd-spd, standby-chw-pump-vfd-pwr, chw-hx-B-out, chw-hx-A-in-mapsto, lead-chw-booster-pump-vfd-fb, lead-chw-booster-pump-in, chw-hx-chw-flow-sensor, lead-chw-pump-vfd-flt, lead-chw-pump-out-mapsto, standby-chw-booster-pump-vfd-pwr, chw-hx-A-out-mapsto, standby-chw-booster-pump-vfd-energy, standby-chw-booster-pump-vfd-cur, lead-chw-booster-pump-vfd-volt, standby-chw-pump-onoff-sts, lead-chw-pump-in, standby-chw-booster-pump-out, standby-chw-pump-out-mapsto, chw-hx-B-in, standby-chw-pump-vfd-cur, lead-chw-pump-onoff-cmd, chw-hx-chw-flow, standby-chw-pump-in-mapsto, standby-chw-booster-pump-vfd-volt, standby-chw-booster-pump, lead-chw-pump-

In [11]:
# fill in all the extra parameters with invented names
for templ in things:
    if isinstance(templ, Graph):
        bldg.add_graph(templ) # template is already complete!
        continue
    print(templ.name)
    _, graph = templ.fill(BLDG, include_optional = True) # invent URIs for things we don't need names for (like ducts)
    bldg.add_graph(graph)

makeup-air-unit
vav-reheat
duct
vav-reheat
duct
hvac-space
duct
fcu
duct
hvac-space
duct
chilled-water-system
hot-water-system


In [12]:
# look at finished model
print(bldg.graph.serialize())
bldg.graph.serialize("output.ttl")

@prefix bldg: <urn:ex/> .
@prefix ns1: <http://data.ashrae.org/standard223#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix qudtqk: <http://qudt.org/vocab/quantitykind/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

ns1:EnumerationKind-Default a ns1:EnumerationKind-AlarmStatus,
        ns1:EnumerationKind-Override ;
    rdfs:label "AlarmStatus-Alarm"@en,
        "Override-Default"@en .

ns1:EnumerationKind-Overridden a ns1:EnumerationKind-AlarmStatus,
        ns1:EnumerationKind-Override ;
    rdfs:label "AlarmStatus-Ok"@en,
        "Override-Overridden"@en .

bldg: a owl:Ontology .

bldg:CHWS a ns1:System ;
    rdfs:label "Chilled Water System" ;
    ns1:contains bldg:bypass-valve_c090baa0,
        bldg:chw-hx_1dce3a01,
        bldg:lead-chw-booster-pump_a3764022,
        bldg:lead-chw-pump_0d3bc890,
        bldg:standby-chw-b

<Graph identifier=6acacd9a-48bb-4e02-9f22-294dc0ccb0a0 (<class 'rdflib.graph.Graph'>)>

In [17]:
# validate against 223P
ctx = bldg.validate([s223.get_shape_collection()])
print(ctx.valid)
print(ctx.report_string)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [18]:
bldg.add_graph(mau_templ.fill(BLDG)[1])

In [19]:
ctx = bldg.validate([s223.get_shape_collection()])
print(ctx.valid)
print(ctx.report_string)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
print(mau_templ.fill(BLDG)[1].serialize())


@prefix ns1: <http://data.ashrae.org/standard223#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix qudtqk: <http://qudt.org/vocab/quantitykind/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

ns1:EnumerationKind-Default a ns1:EnumerationKind-AlarmStatus ;
    rdfs:label "AlarmStatus-Alarm"@en .

ns1:EnumerationKind-Overridden a ns1:EnumerationKind-AlarmStatus ;
    rdfs:label "AlarmStatus-Ok"@en .

<urn:ex/name_98b999ad> a ns1:AirHandlingUnit ;
    ns1:contains <urn:ex/MAU-HRC_89c97942>,
        <urn:ex/cooling-coil_5a4aab11>,
        <urn:ex/evaporative-cooler_b0e28a06>,
        <urn:ex/final-filter_ec8fd069>,
        <urn:ex/heating-coil_5a2496df>,
        <urn:ex/oad_c6b3cac9>,
        <urn:ex/pre-filter_b4aedb5a>,
        <urn:ex/sa_pressure_sensor_775596e6>,
        <urn:ex/supply-fan_b376218c> ;
    ns1:hasConnectionPoint <urn:ex/air-supply_def930cc>,
        <urn:ex/out